In [1]:
%pip install scikit-learn
import pandas as pd
from sklearn.model_selection import train_test_split

# Load and inspect with delimiter handling
try:
    df = pd.read_csv("../data/MachineLearningRating_v3.txt", delimiter='\t')
except Exception as e:
    print("Error loading file:", e)
    try:
        df = pd.read_csv("../data/MachineLearningRating_v3.txt", delimiter=',')
    except Exception as e2:
        print("Fallback load also failed:", e2)
        df = None

if df is not None:
    # Split the single column into multiple columns if needed
    if df.shape[1] == 1:
        df = df[df.columns[0]].str.split('|', expand=True)
        header = [
            'UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims'
        ]
        df.columns = header
        if df.iloc[0].equals(pd.Series(header)):
            df = df.iloc[1:].reset_index(drop=True)
    # Convert numeric columns
    for col in ['TotalClaims', 'NumClaims']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # Filter for severity prediction (only with claims)
    severity_df = df[df['TotalClaims'] > 0].copy()
    # Fill or drop missing values
    df.fillna(method='ffill', inplace=True)
    # Feature engineering
    df['ClaimOccurred'] = (df['TotalClaims'] > 0).astype(int)
    if 'NumClaims' in df.columns:
        df['ClaimSeverity'] = df['TotalClaims'] / df['NumClaims'].replace(0, 1)
    else:
        df['ClaimSeverity'] = float('nan')
    # Encode only selected categorical variables to avoid memory issues
    categorical_cols = [
        'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
        'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone',
        'ItemType', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'bodytype', 'TermFrequency',
        'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType'
    ]
    # Only keep columns that exist in df
    categorical_cols = [col for col in categorical_cols if col in df.columns]
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    # Define features and target
    X_sev = severity_df.drop(columns=["TotalClaims"])
    y_sev = severity_df["TotalClaims"]
    X_clf = df.drop(columns=["TotalClaims"])
    y_clf = df["ClaimOccurred"]
    # Split
    X_train_sev, X_test_sev, y_train_sev, y_test_sev = train_test_split(X_sev, y_sev, test_size=0.2, random_state=42)
    X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)
else:
    print("DataFrame could not be loaded.")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\rebika\AppData\Local\Temp\ipykernel_2696\470017736.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\rebika\AppData\Local\Temp\ipykernel_2696\470017736.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [4]:
%pip install xgboost
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor 
from sklearn.metrics import mean_squared_error, r2_score

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42)
}

for name, model in models.items():
    model.fit(X_train_sev, y_train_sev)
    preds = model.predict(X_test_sev)
    print(f"{name} RMSE: {mean_squared_error(y_test_sev, preds, squared=False):.2f}")
    print(f"{name} R²: {r2_score(y_test_sev, preds):.2f}")


^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
%pip install xgboost
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier # type: ignore
from sklearn.metrics import classification_report, roc_auc_score

clf_models = {
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "XGBoost Classifier": XGBClassifier(random_state=42)
}

for name, model in clf_models.items():
    model.fit(X_train_clf, y_train_clf)
    preds = model.predict(X_test_clf)
    print(f"{name} AUC: {roc_auc_score(y_test_clf, preds):.2f}")
    print(classification_report(y_test_clf, preds))


In [ ]:
%pip install shap
import shap

# Use SHAP for best performing regression model
best_model = XGBRegressor().fit(X_train_sev, y_train_sev)
explainer = shap.Explainer(best_model)
shap_values = explainer(X_test_sev)

shap.summary_plot(shap_values, X_test_sev, plot_type="bar")
